<a href="https://colab.research.google.com/github/Gustavo-Pinho/p1-predictive/blob/main/Exec_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install ucimlrepo

In [14]:
# Importando bibliotecas necessárias
from ucimlrepo import fetch_ucirepo
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [15]:
# Buscando o dataset
real_estate_valuation = fetch_ucirepo(id=477)

# Dados como dataframes pandas
X = real_estate_valuation.data.features  # Features
y = real_estate_valuation.data.targets    # Target (preço por unidade de área)

In [16]:
# Exibindo metadados e informações das variáveis (opcional)
print("Metadados:")
print(real_estate_valuation.metadata)
print("\nInformações das variáveis:")
print(real_estate_valuation.variables)

Metadados:
{'uci_id': 477, 'name': 'Real Estate Valuation', 'repository_url': 'https://archive.ics.uci.edu/dataset/477/real+estate+valuation+data+set', 'data_url': 'https://archive.ics.uci.edu/static/public/477/data.csv', 'abstract': 'The real estate valuation is a regression problem. The market historical data set of real estate valuation are collected from Sindian Dist., New Taipei City, Taiwan. ', 'area': 'Business', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 414, 'num_features': 6, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Y house price of unit area'], 'index_col': ['No'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2018, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C5J30W', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 373, 'type': 'NATIVE', 'title': 'Building real estate valuation models with comparative approach through case-based reasoning', 'author

In [17]:
# Pré-processamento: Padronizando as features (importante para Lasso e Ridge)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [18]:
# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [19]:
# Instanciando os modelos
lasso = Lasso(alpha=1.0)  # alpha controla a força da regularização L1
ridge = Ridge(alpha=1.0)  # alpha controla a força da regularização L2

In [20]:
# Avaliando com validação cruzada (5 folds) no conjunto de treino
lasso_cv_scores = cross_val_score(lasso, X_train, y_train.values.ravel(), cv=5, scoring='r2')
ridge_cv_scores = cross_val_score(ridge, X_train, y_train.values.ravel(), cv=5, scoring='r2')

In [21]:
# Treinando os modelos no conjunto de treino completo
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)

Ridge()

In [22]:
# Avaliando no conjunto de teste
lasso_test_score = lasso.score(X_test, y_test)
ridge_test_score = ridge.score(X_test, y_test)

In [23]:
# Resultados
print("\nResultados da Validação Cruzada (R² médio):")
print(f"Lasso: {lasso_cv_scores.mean():.4f} (± {lasso_cv_scores.std():.4f})")
print(f"Ridge: {ridge_cv_scores.mean():.4f} (± {ridge_cv_scores.std():.4f})")


Resultados da Validação Cruzada (R² médio):
Lasso: 0.5205 (± 0.0841)
Ridge: 0.5398 (± 0.0863)


In [24]:
print("\nDesempenho no Conjunto de Teste (R²):")
print(f"Lasso: {lasso_test_score:.4f}")
print(f"Ridge: {ridge_test_score:.4f}")


Desempenho no Conjunto de Teste (R²):
Lasso: 0.6674
Ridge: 0.6814


In [25]:
# Comparando os coeficientes (para interpretabilidade)
lasso_coefs = pd.Series(lasso.coef_, index=X.columns)
ridge_coefs = pd.Series(ridge.coef_, index=X.columns)
print("\nCoeficientes do Lasso:")
print(lasso_coefs)
print("\nCoeficientes do Ridge:")
print(ridge_coefs)


Coeficientes do Lasso:
X1 transaction date                       0.467370
X2 house age                             -2.014102
X3 distance to the nearest MRT station   -5.170648
X4 number of convenience stores           2.649982
X5 latitude                               2.335047
X6 longitude                              0.000000
dtype: float64

Coeficientes do Ridge:
X1 transaction date                       1.525186
X2 house age                             -3.071378
X3 distance to the nearest MRT station   -5.935637
X4 number of convenience stores           3.214959
X5 latitude                               2.845204
X6 longitude                             -0.405451
dtype: float64


A regressão linear, mesmo com regularização (Lasso e Ridge), assume uma relação linear entre as variáveis e o target (preço), o que pode ser uma simplificação excessiva. Quando as relações entre as variáveis e o preço são não lineares, esses modelos não são tão eficazes. A linearidade pode não ser suficiente para capturar toda a complexidade dos dados, levando a um desempenho limitado.

Para melhorar os resultados, seria interessante o ajustes nos hiperparâmetros (como o valor de alpha no Lasso e Ridge) e uma análise mais aprofundada das características dos dados podem ajudar a melhorar o desempenho dos modelos.